In [18]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Una classe sulle classi di Python

## Concetti di base di Python

Tutto in Python è un oggetto con un'**identità** e un **tipo**!

Puoi accedervi utilizzando le funzioni `id()` e `type()`.

Python fornisce già tipi incorporati (None, int, str, ...) che storicamente non potevano essere estesi.

**In Python 3, tutti i tipi sono anche classi**, i termini vengono usati in modo intercambiabile e puoi ereditare da tutto.

### Terminologia


Dichiarazioni di Classe

In [1]:
class MyClass:
    pass

Creare un'istanza di una Classe

In [21]:
obj = MyClass()
print(type(obj))

<class '__main__.MyClass'>


Quali sono i built-in types?

In [3]:
age = 50
print(type(age))

<class 'int'>


Quindi una dichiarazione di intero restituisce effettivamente un'istanza della classe int.

In [22]:
age = int(50)

### Quanto e' profonda la tana del coniglio?

La classe `int` ha una classe base?

In [5]:
print(int.__bases__)

(<class 'object'>,)


E `object` ha una base class?

In [6]:
print(object.__bases__)

()


La classe `object` e' la base di tutte le classi in Python3

In [7]:
class MyClass:
    pass

# is equivalent to

class MyClass(object):
    pass

Tutto in Python e' un oggetto!

Se tutto è un oggetto, quale è il tipo di una dichiarazione di classe?

In [23]:
print(type(int))
print(type(MyClass))
print(type(object))

<class 'type'>
<class 'type'>
<class 'type'>


Le dichiarazioni di classe sono istanze della classe `type`, che a sua volta è un'istanza di `type` e una sottoclasse della classe `object`.

In [24]:
print(type(type))
print(type.__bases__)

<class 'type'>
(<class 'object'>,)


`type` è una cosiddetta **metaclass**, una classe le cui istanze sono classi.

### Dichiarazione dinamica di classe

invece di un solo argomento, `type` può essere chiamato con tre parametri:

`type(nome_classe, superclassi, dizionario_attributi)`

In [10]:
MyClass = type('MyClass', (), {})
obj = MyClass()
print(type(obj))

<class '__main__.MyClass'>


## Torniamo a qualcosa di utile - come posso migliorare le mie classi

- fondamentalmente le classi sono costruttori di oggetti
- permettono di definire oggetti personalizzati, raggruppando dati e funzionalità
- simili alle strutture/classi in c++ (la differenza è l'accesso pubblico/privato predefinito)
- in python tutte le funzioni sono virtuali

### Anatomia di una classe

In [30]:
class MyClass:
    """Un semplice esempio di classe"""
    unmutable = 100 # class attribute shared by each instance
    mutable = []    # be careful with mutable class attributes (avoid this!)
   
    def set_instance_attr(self, value): # instance method, despite the name not unique to each instance
        """Assign to an instance attribute"""
        self.instance_attr = value # instance attribute unique to each instance

# Create two instances of MyClass
obj_1 = MyClass()
obj_2 = MyClass()
# This example shows what goes wrong with mutable class attributes
obj_2.mutable.append('Attenzione!')
print(obj_1.mutable)
print(obj_1.unmutable)
print(obj_2.unmutable)

obj_1.set_instance_attr(4)
obj_1.instance_attr
obj_2.instance_attr

['Attenzione!']
100
100


AttributeError: 'MyClass' object has no attribute 'instance_attr'

In [33]:
obj_1.first_name = 'Alex'
obj_1.first_name

obj_2.first_name

AttributeError: 'MyClass' object has no attribute 'first_name'

### Attributi

- tutti gli attributi/attributi di istanza di una classe/istanza sono memorizzati in un dizionario.

In [12]:
print('Class attributes:', MyClass.__dict__)
print('Instance attributes:', obj_1.__dict__)

Class attributes: {'__module__': '__main__', '__doc__': 'Un semplice esempio di classe', 'unmutable': 100, 'mutable': ['Attenzione!'], 'set_instance_attr': <function MyClass.set_instance_attr at 0x110be7700>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>}
Instance attributes: {}


- i dizionari occupano molta memoria rispetto alle liste
- se non hai bisogno di aggiungere attributi alle tue istanze dinamicamente, puoi utilizzare `__slots__`
- gli attributi `__slots__` hanno anche un accesso più veloce 

In [13]:
import sys

class MyClassWithSlots:
    """A simple example class using slots."""
    __slots__ = ['instance__attr']
    unmutable = 100
    mutable = []

    def set_instance_attr(self, value):
        """Assign to an instance attribute"""
        self.instance_attr = value

# Compare the size of classes, the one using slots is smaller 
obj_slots = MyClassWithSlots()
print('Size of standard class:', sys.getsizeof(obj_1))
print('Size of class using slots:', sys.getsizeof(obj_slots))
# However you loose the ability to assign instance attributes dynamically
obj_slots.new_attr = 42

Size of standard class: 48
Size of class using slots: 40


AttributeError: 'MyClassWithSlots' object has no attribute 'new_attr'

- puoi comunque permettere attributi di istanza dinamici aggiungendo `'__dict__'` a `__slots__`

### Attributi privati e metodi

- single underscore indica che gli attributi/metodi sono intesi come privati (except wildcard imports)
- double underscore `__var` e' sostituito da `_classname__var` (name mangling)

In [14]:
class MyClass:
    """An example class with private attributes and methods"""
    __private_attr = 'You see my private attribute!'
    
    def __private_method(self):
        """A private method"""
        print('You see my private method!')

obj = MyClass()
# You can still acces private attributes by using the mangled name
print(obj._MyClass__private_attr)
obj._MyClass__private_method()
# However, accessing it directly will give an error
print(obj.__private_attr)

You see my private attribute!
You see my private method!


AttributeError: 'MyClass' object has no attribute '__private_attr'

## Alcuni metodi decoratori utili delle classi

### @property decorator - come convertire un metodo in un attributo

Modificare le variabili di istanza non cambia le altre variabili di istanza.




In [35]:
class Person:
    
    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname
        self.fullname = self.first + ' '+ self.last
    
person = Person('Alex', 'Saro')
print(person.fullname)
person.first = 'Alexandro'
print(person.fullname, person.first)


Alex Saro
Alex Saro Alexandro


Il decoratore @property permette di continuare ad utilizzare i metodi come attributi per non interrompere il codice esistente aggiornando automaticamente le quantita'.






In [20]:
class Person:

    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname
    
    @property
    def fullname(self):
        return self.first + ' '+ self.last

person = Person('Alex', 'Saro')
print(person.fullname)
person.first = 'Alexandro'
print(person.fullname)

Alex Saro
Alexandro Saro


# Esercizio: Calcolo della Magnitudine Apparente di una Stella

In questo esercizio, implementeremo una classe `Star` per rappresentare una stella e calcolare dinamicamente la sua **magnitudine apparente**. La magnitudine apparente di una stella dipende sia dalla sua **magnitudine assoluta** che dalla sua **distanza dalla Terra**. 

#### Dettagli

1. La **magnitudine assoluta** (\( M \)) è la luminosità intrinseca della stella, ovvero la luminosità osservabile se la stella si trovasse a una distanza standard di 10 parsec dalla Terra.
2. La **magnitudine apparente** (\( m \)) rappresenta come appare luminosa la stella vista dalla Terra e dipende dalla distanza effettiva della stella.

#### Formula

La relazione tra magnitudine apparente, magnitudine assoluta e distanza \( d \) (in parsec) è:

$$
m = M + 5 \cdot \log_{10}\left(\frac{d}{10}\right)
$$

Dove:
- \( m \) è la **magnitudine apparente**,
- \( M \) è la **magnitudine assoluta**,
- \( d \) è la **distanza** in parsec.

#### Obiettivi dell’esercizio

1. Creare una classe `Star` con i seguenti attributi:
   - `name`: Nome della stella.
   - `absolute_magnitude`: Magnitudine assoluta della stella.
   - `distance`: Distanza della stella dalla Terra (in parsec).
   - `apparent_magnitude`: Magnitudine apparente calcolata dinamicamente.

2. Implementare `apparent_magnitude` come una **proprietà dinamica** (`@property`), in modo che il suo valore si aggiorni automaticamente ogni volta che `absolute_magnitude` o `distance` vengono modificati.

#### Esempio di utilizzo
Dopo aver implementato la classe, possiamo creare una stella e modificarne la distanza o la magnitudine assoluta per vedere come cambia la magnitudine apparente:

```python
# Esempio di utilizzo della classe
star = Star('Sirius', 1.4, 2.6)
print(f"{star.name} apparent magnitude: {star.apparent_magnitude:.2f}")

# Cambiare la distanza
star.distance = 5.0
print(f"{star.name} apparent magnitude at 5 parsecs: {star.apparent_magnitude:.2f}")

# Cambiare la magnitudine assoluta
star.absolute_magnitude = 1.0
print(f"{star.name} apparent magnitude with absolute magnitude of 1.0: {star.apparent_magnitude:.2f}")



inoltre, permette l'impostazione personalizzata e l'eliminazione degli attributi.

In [40]:
class Person:

    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname

    @property
    def fullname(self):
        return self.first + ' ' + self.last
     
    @fullname.setter
    def fullname(self, name):
        firstname, lastname = name.split()
        self.first = firstname
        self.last = lastname
        
    @fullname.deleter
    def fullname(self):
        self.first = 'None'
        self.last = 'None'  

In [41]:
person = Person('Alex', 'Saro')
print(person.fullname)
# Set a new name
person.fullname = 'Matteo Costanzi'
# Other attributes get automatically adjusted
print(person.fullname)
print(person.first)
print(person.last)
# Delete name
del person.fullname
print(person.fullname)

Alex Saro
Matteo Costanzi
Matteo
Costanzi
None None


usa le proprietà per un'incapsulazione completa dove puoi definire regole su come gli attributi privati possono essere ottenuti e impostati.

In [44]:
class Encapsulated:
    
    def __init__(self, name):
        self.private_attr = name # This assignment calls the setter method
        
    @property
    def private_attr(self):
        return self.__private_attr
    
    @private_attr.setter
    def private_attr(self, name):
        if len(name) > 0 and len(name) < 10:
            self.__private_attr = name
        else:
            self.__private_attr = None
        
obj = Encapsulated('Alex Saro')
# The name is too long
print(obj.private_attr)
obj = Encapsulated('Alexandro Saro')
# The name is too long
print(obj.private_attr)

Alex Saro
None


### Decoratore @staticmethod - come chiamare metodi senza istanziare

- i metodi di istanza possono essere chiamati solo da istanze della classe
- il decoratore @staticmethod lo converte in un metodo statico
- i metodi statici possono anche essere chiamati senza prima istanziare la classe
- i metodi statici non conoscono altri attributi
- si potrebbe usare una funzione normale, tuttavia in questo modo è logicamente contenuta nella classe per leggibilità.

In [46]:
class MyClass:
  
       @staticmethod
       def find_max(number):
           return max(number, 42)

# You can use the find_max method using the class directly
print(MyClass.find_max(40))
# or using an instance
obj = MyClass()
print(obj.find_max(101))

42
101


### Decoratore @classmethod - un metodo statico che conosce la sua classe

- Il decoratore @classmethod converte un metodo di istanza in un metodo di classe
- simile ai metodi statici, possono essere chiamati senza prima istanziare la classe
- prende la classe come parametro: `cls`
- i metodi di classe conoscono gli altri attributi
- i metodi di classe permettono costruttori predefiniti (funzioni di fabbrica)

In [48]:
class MyClass:
    other_number = 42
           
    @classmethod
    def find_max(cls, number):
        return max(number, cls.other_number)
    
print(MyClass.find_max(40))
obj = MyClass()
print(obj.find_max(101)) # This would also work with normal instance method

42
101


In [53]:
class Pizza:
    """This class shows the concept of factory functions"""
    
    def __init__(self, ingredients):
        self.ingredients = ingredients

    def __repr__(self):
        return f'Pizza({self.ingredients})'

    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'pomodoro'])

    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'pomodoro', 'prosciutto'])

print(Pizza.margherita()
Pizza.prosciutto()

Pizza(['mozzarella', 'pomodoro'])


Pizza(['mozzarella', 'pomodoro', 'prosciutto'])

## Metodi speciali - aggiungere funzionalità alle classi

### Esempio: Cosa succede quando creiamo un'istanza?





In [56]:
obj = MyClass()

- il metodo `__call__` della classe genitore viene chiamato, in questo caso il metodo `type_call` della metaclass `type`
- questo a sua volta invoca i metodi `__new__` e `__init__` della classe stessa
- tutti questi tipi di metodi sono chiamati metodi **speciali**, **magici** o **dunder** e sono il modo di Python di sovraccaricare gli operatori.

### `__new__` - modifica come vengono create le istanze della classe

- chiamato automaticamente quando una classe viene istanziata
- crea effettivamente l'oggetto
- utilizzato per personalizzare come vengono create le nuove istanze di classe
- è qui che gli oggetti immutabili vengono inizializzati poiché qualsiasi attributo impostato in `__init__` può essere modificato.


In [57]:
class Singleton:
    """This is an example of how to implement a singleton"""
    __instance = None
    
    def __new__(cls): # As in the case with self, cls is a naming convention
        if not cls.__instance:
            cls.__instance = object.__new__(cls)
            return cls.__instance
        else:
            print('An instance already exists!')
        
single_one = Singleton()
print(single_one)
single_two = Singleton()
print(single_two)

An instance already exists!
None


### `__init__` - il costruttore/inizializzatore

- chiamato automaticamente quando una classe viene istanziata
- utilizzato per inizializzare gli attributi.


### `__repr__` - la rappresentazione formale

- chiamato da repr()
- dovrebbe poter agire come argomento per eval() e restituire lo stesso oggetto
- dovrebbe sempre essere implementato.

In [66]:
class Person:
    
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __repr__(self):
        """Is called by the repr() function"""
        return (f'Person("{self.first}","{self.last}",{self.age})')

person = Person('Alex', 'Saro', 43)
# This should return itself
print(eval(repr(person)))

Person("Alex","Saro",43)


### `__str__` - la rappresentazione come stringa

- chiamato da str() e dall'istruzione print
- è la rappresentazione leggibile umana
- se non implementato, str() restituisce il risultato del metodo `__repr__`
- quest'ultimo avviene sempre anche per gli oggetti nei contenitori.

In [67]:
class Person:
    
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __repr__(self):
        return f'Person("{self.first}","{self.last}",{self.age})'

    def __str__(self):
        """Is called by the str() and print() function"""
        return f'{self.first} {self.last}, {self.age}'
    
person = Person('Alex', 'Saro', 43)
# Print a nice human readable version
print(person)

Alex Saro, 43


### `__del__` - il distruttore/finalizzatore

- chiamato quando il garbage collector sta eliminando oggetti senza riferimenti.

In [87]:
class MyClass:
    
    def __del__(self):
        """Is called when the garbage collector frees up the memory"""
        print(f'{self.__class__.__name__} class destroyed!')
        
    def __str__(self):
        return "I exist!"
        
obj = MyClass()
reference = obj
print(obj, reference)
del obj
print(reference)
del reference

I exist! I exist!
I exist!
MyClass class destroyed!


In [88]:
class MyClass:

    def __enter__(self):
        """Gets called when with statement is entered"""
        print('Entering with statement!')
        
    def __exit__(self, exc_type, exc_value, traceback):
        """Gets called when with statement is left"""
        print('This is always garanteed to be called!')

with MyClass() as example:
    print('Executing code!')

Entering with statement!
Executing code!
This is always garanteed to be called!


### `__getitem__`, `__setitem__`, `__len__` - making your class a container

In [93]:
class MyContainer:
    
    def __init__(self):
        self.data = []
        
    def __len__(self):
        return len(self.data)
    
    def append(self, item):
        self.data.append(item)
    
    def __getitem__(self, sliced):
        return self.data[sliced]
    
    def __setitem__(self, key, item):
        self.data[key] = item
        
container = MyContainer()
container.append("First")
container.append("Second")
container.append("Third")
print('First entry:', container[0])
print('Length:', len(container))
print("Second entry:", container[1])
container[1] = 2
print('Second entry', container[1])

First entry: First
Length: 3
Second entry: Second
Second entry 2


In [95]:
class MyContainer:
    
    def __init__(self):
        self.data = []
        
    def __len__(self):
        return len(self.data)
    
    def append(self, item):
        self.data.append(item)
    
    def __getitem__(self, sliced):
        return self.data[sliced]
    
    def __setitem__(self, key, item):
        self.data[key] = item
        
container = MyContainer()
container.append("First")
container.append("Second")
container.append("Third")
print('First entry:', container[0])
print('Length:', len(container))
print("Second entry:", container[1])
container[1] = 2
print('Second entry', container[1])

First entry: First
Length: 3
Second entry: Second
Second entry 2


### `__eq__`, `__le__`, `__lt__`, `__ne__`, `__ge__`, `__gt__` - confronto tra due oggetti

- metodi di "confronto avanzato"
- `__eq__` di default confronta utilizzando `is`
- è necessario implementare solo uno di ciascun tipo.


In [101]:
class Person:
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __le__(self, other):
        return self.age <= other.age

    def __lt__(self, other):
        return self.age < other.age
    
person1 = Person('Alex', 'Saro', 43)
person2 = Person('Matteo', 'Costanzi', 37)
print(person1 != person2)
print(person1 <= person2)
print(person1 > person2)

True
False
True


- utilizzando `total_ordering` devi implementare solo uno tra `__lt__`, `__le__`, `__gt__` o `__ge__`
- se non implementi `__eq__`, gli oggetti verranno comunque confrontati solo utilizzando `is`.

In [102]:
from functools import total_ordering

@total_ordering
class Person:
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __lt__(self, other):
        return self.age < other.age
    
    def __repr__(self):
        return f'Person("{self.first}","{self.last}",{self.age})'
    
person1 = Person('Alex', 'Saro', 43)
person2 = Person('Matteo', 'Costanzi', 37)
print(person1 != person2)
print(person1 <= person2)
print(person1 < person2)

True
False
False


### `__hash__` - rendere la tua classe "hashable" (in grado di essere hashata???)

se `__eq__` non è definito:
- ogni istanza separata avrà un hash diverso
- solo un uso limitato come hash, poiché gli oggetti creati con gli stessi valori hanno hash diversi
- non dovresti nemmeno definire un metodo `__hash__`

se `__eq__` è definito ma non `__hash__`:
- le istanze non potranno essere utilizzate come elementi in collezioni hashabili

se una classe definisce oggetti mutabili, non dovrebbe implementare `__hash__`, **solo gli oggetti immutabili dovrebbero essere utilizzati come hash**.

In [103]:
# Our previously defined persons cannot be used as keys in dictionaries
dic = {person1: 1, person2: 2}

TypeError: unhashable type: 'Person'

In [7]:
class Person:
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __repr__(self):
        return f'Person("{self.first}","{self.last}",{self.age})'

    def __hash__(self):
        """The objects of the class are mutable!"""
        return hash(tuple([self.first, self.last, self.age]))
    
person1 = Person('Alex', 'Saro', 43)
person2 = Person('Alex', 'Saro', 43)
print(hash(person1) == hash(person2))
dic = {person1: 1}
# Everything seems to work, person2 is an accepted key 
print('dic[person2]:', dic[person2])
# We change person1 and both person1 and person2 will give a key error
person1.age = 42
print(dic[person1])


True
dic[person2]: 1


KeyError: Person("Alex","Saro",42)

- quando abbiamo modificato person1, abbiamo anche modificato la chiave del dizionario
- l'hash non si riferirà più al bucket corretto nel dizionario per quella chiave
- l'unico modo per utilizzare una classe mutabile è quando l'hash si basa sull'identità e non sul valore.

## Dataclass

- il decoratore `@dataclass` converte una classe in una dataclass
- aggiunge automaticamente `__init__`, `__repr__` e `__eq__`
- funziona ancora come una classe regolare.

In [9]:
class Person:

    def __init__(self, firstname: str='unknown', lastname: str='unknown', age: int=-1): # These are variabel annotations or type hints
        self.first = firstname
        self.last = lastname
        self.age = age
        
    def __repr__(self):
        return f'Person(first={self.first}, last={self.last}, age={self.age})'
    
    def __eq__(self, other):
        return (self.first, self.last, self.age) == (other.first, other.last, other.age)

In [12]:
from dataclasses import dataclass

@dataclass
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1

person1 = DataClassPerson('Alex', 'Saro', 43)
person2 = DataClassPerson('Alex', 'Saro', 43)
print(person1)
person1 == person2

DataClassPerson(first='Alex', last='Saro', age=43)


True

I suggerimenti di type sono obbligatori per le dataclasses, `any` e' anche un opzione.

- le righe sotto la dichiarazione della classe definiscono oggetti di campo, che sono speciali per le dataclass e funzionano come attributi regolari
- i tipi mutabili non sono ammessi, tranne quelli creati dall'opzione `default_factory`.

In [16]:
from dataclasses import dataclass, field

def return_list() -> list:
    return [0, 1, 2]

@dataclass
class DataClassMutable:
    mutable: list = field(default_factory=return_list)
        
obj = DataClassMutable()
print(obj.mutable)

[0, 1, 2]


I campi hanno le seguenti opzioni:

- **default**: valore predefinito del campo
- **default_factory**: funzione che restituisce il valore iniziale del campo
- **init**: utilizza il campo nel metodo `__init__`, il valore predefinito è True
- **repr**: utilizza il campo nella rappresentazione (`repr`) dell'oggetto, il valore predefinito è True
- **compare**: include il campo nei confronti, il valore predefinito è True
- **hash**: include il campo nel calcolo dell'hash, il valore predefinito è utilizzare lo stesso dell'opzione `compare`
- **metadata**: una mappatura con informazioni sul campo.


In [19]:
from dataclasses import dataclass, field, fields

@dataclass
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = field(default=-1, metadata={'unit': 'years'})
        
fields(DataClassPerson)

(Field(name='first',type=<class 'str'>,default='unknown',default_factory=<dataclasses._MISSING_TYPE object at 0x7ff564943be0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 Field(name='last',type=<class 'str'>,default='unknown',default_factory=<dataclasses._MISSING_TYPE object at 0x7ff564943be0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 Field(name='age',type=<class 'int'>,default=-1,default_factory=<dataclasses._MISSING_TYPE object at 0x7ff564943be0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'unit': 'years'}),_field_type=_FIELD))

Il decoratore `@dataclass` può anche essere chiamato con i seguenti parametri:

- **init**: aggiunge il metodo `__init__`, il valore predefinito è True
- **repr**: aggiunge il metodo `__repr__`, il valore predefinito è True
- **eq**: aggiunge il metodo `__eq__`, il valore predefinito è True
- **order**: aggiunge i metodi di ordinamento, il valore predefinito è False
- **unsafe_hash**: forza l'aggiunta di un metodo `__hash__`, il valore predefinito è False
- **frozen**: se True, l'assegnazione ai campi (**e solo ai campi**) solleva un'eccezione, il valore predefinito è False.

`order=True` confronta gli oggetti come se fossero tuple.

In [23]:
from dataclasses import dataclass

@dataclass(order=True)
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1
                                   
person1 = DataClassPerson('Alex', 'Saro', 43)
person2 = DataClassPerson('Matteo', 'Costanzi', 37)
person1 > person2

False

devi impostare un indice di ordinamento per ottenere un ordinamento personalizzato.

In [25]:
from dataclasses import dataclass, field

@dataclass(order=True)
class DataClassPerson:
    sort_index: int = field(init=False, repr=False)
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1

    def __post_init__(self): # This is a special method of dataclasses, called by __init__
        self.sort_index = self.age

person1 = DataClassPerson('Alex', 'Saro', 43)
person2 = DataClassPerson('Matteo', 'Costanzi', 37)

person1 > person2

True

`frozen=True` protegge gli attributi dalle modifiche.

In [26]:
@dataclass(frozen=True)
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1
           
person1 = DataClassPerson('Alex', 'Saro', 43)
person1.first = 'Alexandro'

FrozenInstanceError: cannot assign to field 'first'

tuttavia, gli oggetti mutabili possono ancora essere modificati.

In [27]:
def return_list():
    return [0, 1, 2]

@dataclass(frozen=True)
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1
    mutable: list = field(default_factory=return_list)
        
person1 = DataClassPerson('Alex', 'Saro', 43)
person1.mutable[0] = 42
print(person1.mutable)

[42, 1, 2]


## Conclusioni

- tutto in python è un oggetto
- usa le **proprietà** per una migliore impostazione/recupero degli attributi e insieme alle variabili private per un'encapsulazione completa
- i **metodi speciali** possono ampliare notevolmente l'usabilità delle classi; implementa solo quelli di cui hai bisogno
- le **dataclassi** possono facilitare notevolmente la creazione di classi.

In [19]:
class Star:
    def __init__(self, name, absolute_magnitude, distance):
        self.name = name
        self.absolute_magnitude = absolute_magnitude
        self.distance = distance  # distanza in parsec

    @property
    def apparent_magnitude(self):
        # Calcola la magnitudine apparente usando la formula astrofisica
        return self.absolute_magnitude + 5 * np.log10(self.distance / 10)

# Creare un oggetto di esempio e testare il comportamento
star = Star('Sirius', 1.4, 2.6)
print(f"{star.name} apparent magnitude: {star.apparent_magnitude:.2f}")

# Modificare la distanza e verificare che apparent_magnitude si aggiorni automaticamente
star.distance = 5.0
print(f"{star.name} apparent magnitude at 5 parsecs: {star.apparent_magnitude:.2f}")

# Modificare la magnitudine assoluta e verificare l'aggiornamento
star.absolute_magnitude = 1.0
print(f"{star.name} apparent magnitude with absolute magnitude of 1.0: {star.apparent_magnitude:.2f}")


Sirius apparent magnitude: -1.53
Sirius apparent magnitude at 5 parsecs: -0.11
Sirius apparent magnitude with absolute magnitude of 1.0: -0.51
